In [3]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="deepseek-r1:8b")
response=model.invoke("Come up with 10 names for a song about parrots")

In [4]:
print(response)

<think>
Okay, so I need to come up with 10 names for a song about parrots. Hmm, let's see... Parrots are colorful, talkative birds often kept as pets. They can mimic speech and sounds, which might be a fun element in the song.

First, I should think about words related to parrots: parrot, bird, friend, colors like blue, green, yellow, maybe terms like "squawk" or "tweet." Maybe also things like flock, mate, companion.

Let me brainstorm some combinations. How about "Parrot's Lullaby"? That sounds gentle and suggests a song for a parrot. Or "Tropical Breeze," which gives a sense of the bird's environment.

I should consider the rhythm and how the names sound together. Maybe something catchy that people can remember. Words like "Chatterbox" could work, as it plays on their talkative nature. "Flockstar" sounds like a cool name, maybe for the leader of the flock.

Thinking about nature terms: "Feathered Friend," which is friendly and descriptive. "Rainbow Hues" ties in the vibrant colors. 

In [6]:
import pandas as pd

DF=pd.read_excel("Noticias.xlsx").head(10)
DF

,Columna1,Enlaces,Título,info,contenido,Etiqueta
0,0,https://www.eltiempo.com/agresion-contra-un-op...,Operador de grúa quedó inconsciente tras agres...,El conductor de una moto le lanzó el casco y p...,Las autoridades están buscando al conductor de...,colombia
1,1,https://www.eltiempo.com/archivo/documento/CMS...,"Usaquén, primera en infracciones por mal parqueo",La localidad ocupa el primer lugar en comparen...,"""Los andenes son para los peatones"", reclama e...",archivo
2,2,https://www.eltiempo.com/archivo/documento/CMS...,'Me atracaron y vi un arma que me heló la sang...,Un ciudadano relata cómo cuatro hombres lo rob...,A las 7 de la noche me había quedado de encont...,archivo
3,3,https://www.eltiempo.com/archivo/documento/CMS...,"Escoltas mal estacionados, dolor de cabeza de ...",Las zonas de restaurantes se convierten en par...,Atravesados. Eso es lo que se les pasa por la ...,archivo
4,4,https://www.eltiempo.com/archivo/documento/CMS...,Radicado primer proyecto que autorizaría union...,"El representante de 'la U', Miguel Gómez, dijo...",“Estamos proponiendo la figura de un contrato ...,archivo
5,5,https://www.eltiempo.com/archivo/documento/CMS...,'Regresé para decir que sí se puede pedir just...,"El general Jorge Pineda, ex jefe de Inteligenc...",Valle del Cauca 07:50 p.m. Se han recogido cer...,archivo
6,6,https://www.eltiempo.com/archivo/documento/CMS...,"'Sánchez y Torres, parte del proyecto político...",NaN,Valle del Cauca 07:50 p.m. Se han recogido cer...,archivo
7,7,https://www.eltiempo.com/archivo/documento/CMS...,"Con grupo élite, perseguirán a los abusadores ...",Fiscalía y Policía pondrán a los abusadores se...,Los agresores sexuales serán perseguidos por u...,archivo
8,8,https://www.eltiempo.com/archivo/documento/CMS...,Construirán 100 nuevas casas de emergencia par...,Estas viviendas funcionan como una solución te...,Cien viviendas de emergencia (se construyen en...,archivo
9,9,https://www.eltiempo.com/archivo/documento/CMS...,Aguardiente Néctar y Ron Santafé le apuestan a...,La meta para estos productos es posicionarse e...,"La idea no solo es llegar a más departamentos,...",archivo


In [13]:
from langchain import PromptTemplate

template="""
Eres un asistente experto en noticias. Te voy a solicitar lo siguiente:

Para la siguiente noticia:
{noticia}

vas a extraer lo siguiente:

- Un resumen
- Una lista con las personas
- Una lista con las locaciones geográficas

Responderas todo con un json, con llaves:

-Resumen: <string>
-Personas: Lista de <strings>
-Locaciones: Lista de <strings>
"""

prompt=PromptTemplate(input_variables=["noticia"],
                          template=template)

query=prompt.format(noticia=DF['contenido'].loc[0])

print(query)


Eres un asistente experto en noticias. Te voy a solicitar lo siguiente:

Para la siguiente noticia:
Las autoridades están buscando al conductor de una moto que le lanzó el casco a Carlos Alberto Carmona, operador de grúa de la empresa Segrup , quien perdió por una hora el conocimiento, tras la agresión. En un video quedó registrado el momento en el que tanto el trabajador de la empresa como el motociclista se encuentran discutiendo, cerca de la avenida Villavicencio con Gaitán Cortés. En el acalorado encuentro verbal, ambos amagan con golpearse con los objetos que tienen en sus manos; entonces, el conductor de la moto arroja su casco contra el operador. El hombre recibe el impacto en su cara, por lo que s e desgonza, y en la caída se golpea la cabeza con la grúa que conduce ; el agredido perdió el conocimiento por cerca de una hora. En ese instante, el agresor se retira caminando del lugar, y en repetidas ocasiones mira para atrás, en donde está el operador en el suelo. Luego, su comp

In [14]:
Respuestas=[]
for noticia in DF['contenido']:
    response=model.invoke(prompt.format(noticia=noticia))
    Respuestas.append(response)
    

In [18]:
Respuestas

['<think>\nOkay, I need to help the user extract specific information from their provided news text. The user wants a JSON response containing a summary, a list of people, and a list of geographical locations mentioned in the news.\n\nFirst, I\'ll read through the entire news content to understand the main points. The article is about an incident where a motorbike driver attacked a grúa operator, causing him to lose consciousness. The location mentioned is near Avenida Villavicencio with Gaitán Cortés in Bogotá. Other locations include "Donatón Valle Solidario," but that\'s more of a place name without specific geographical context here.\n\nI need to create a summary that captures the key events: the attack, the consequences, and the authorities\' response. Next, I\'ll list all individuals involved, including Carlos Alberto Carmona and the motorbike driver whose name isn\'t directly provided, but the license plate number might be enough to note as a person of interest.\n\nFor geographi

In [17]:
#Extraemos json por respuesta
import re
import re
import json

for texto in Respuestas:
    # Expresiones regulares para extraer la información
    resumen_patron = re.search(r'"Resumen": "(.*?)"', texto)
    personas_patron = re.search(r'"Personas": \[(.*?)\]', texto)
    locaciones_patron = re.search(r'"Locaciones": \[(.*?)\]', texto)

    # Obtener los valores extraídos
    resumen = resumen_patron.group(1) if resumen_patron else ""
    personas = re.findall(r'"([^"]+)"', personas_patron.group(1)) if personas_patron else []
    locaciones = re.findall(r'"([^"]+)"', locaciones_patron.group(1)) if locaciones_patron else []

    # Crear el diccionario JSON
    resultado = {
        "Resumen": resumen,
        "Personas": personas,
        "Locaciones": locaciones
    }

    # Convertir a JSON
    json_resultado = json.dumps(resultado, indent=2, ensure_ascii=False)

    # Imprimir el JSON
    print(json_resultado)



{
  "Resumen": "Las autoridades buscan al conductor de una moto que agredió a Carlos Alberto Carmona, operador de grúa de la empresa Segrup, quien sufrió un accidente y perdió el conocimiento por cerca de una hora. El incidente ocurrió en Bogotá, cerca de la avenida Villavicencio con Gaitán Cortés, y fue registrado en un video.",
  "Personas": [
    "Carlos Alberto Carmona",
    "Conductor de la moto (JTO 18C)"
  ],
  "Locaciones": [
    "Bogotá",
    "Avenida Villavicencio con Gaitán Cortés"
  ]
}
{
  "Resumen": "En Bogotá, varios sectores de la ciudad enfrentan problemas relacionados con el parqueo de vehículos en zonas críticas, lo que afecta a peatones y residentes. Entre los problemas destacados se encuentran: - La falta de espacios para parquear ambulancias en la calle 140 con carrera 12, lo que obliga a los usuarios a caminar o cruzar la vía. - El parqueo de vehículos particulares en la bahía de la Clínica del Norte, dejando disponibles muy pocos espacios. - La ocupación de zona